In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%pip install cellpose==3.0.8

In [11]:
import os

## SETTINGS
BASE_PATH = "/content/drive/MyDrive/szakmai gyakorlat/cellpose-cell-counting"
DATASETS_PATH = "datasets"
DATASET_FILE = "2024.05.09. Ac-kPHSRN+fibronektin.zip"
OUTPUT_PATH = "/content/result"
MODELS_PATH = "models"
MODEL_NAME = "cyto3_all_annotated"

CHANNELS = [0, 0]
FLOW_THRESHOLD = 0.4
CELLPROB_THRESHOLD = 0.0

## CALCULATED PARAMS
DATASET_PATH = os.path.join(BASE_PATH, DATASETS_PATH, DATASET_FILE)
DATASET_FOLDER = DATASET_FILE.split(".zip")[0]
MODEL_PATH = os.path.join(BASE_PATH, MODELS_PATH, MODEL_NAME)

In [ ]:
!unzip "{DATASET_PATH}" -d "/content"

In [13]:
from cellpose import models
import torch

model = models.CellposeModel(
    pretrained_model=MODEL_PATH,
    gpu=torch.cuda.is_available())

In [14]:
import glob
import cv2

img_paths = glob.glob(DATASET_FOLDER + "/*.jpg")
eval_imgs = []

for path in img_paths:
  img = cv2.imread(path, 0) # grayscale
  eval_imgs.append(img)

In [15]:
masks, _, _ = model.eval(
    eval_imgs,
    channels=CHANNELS,
    flow_threshold=FLOW_THRESHOLD,
    cellprob_threshold=CELLPROB_THRESHOLD)

In [16]:
import csv
import cv2
import numpy as np
from cellpose.utils import masks_to_outlines

def get_outlined_image(img, mask, outline_color=(255, 0, 0)):
  outline = masks_to_outlines(mask) * mask
  bgr_image = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

  for y in range(img.shape[0]):
    for x in range(img.shape[1]):
      if outline[y, x] > 0:
        bgr_image[y, x] = outline_color

  return bgr_image

cnts = {}

if not os.path.exists(OUTPUT_PATH):
  os.makedirs(OUTPUT_PATH)

for img, mask, path in zip(eval_imgs, masks, img_paths):
  file_name = path.split("/")[1]
  cv2.imwrite(os.path.join(OUTPUT_PATH, file_name), get_outlined_image(img, mask))
  cnts[file_name] = np.max(mask)

with open(os.path.join(OUTPUT_PATH, "counts.csv"),'w') as f:
  w = csv.writer(f, delimiter=";")
  w.writerows(cnts.items())

In [ ]:
!zip -r /content/result.zip "{OUTPUT_PATH}"